In [3]:
import pandas as pd
import pandas_datareader.data as web
import yfinance as yf
import requests
import io
from datetime import datetime
import warnings
import os

start_date = "2017-01-01"
# end_date = datetime.today().strftime('%Y-%m-%d')
end_date = "2025-10-31" # Because Fama-Factors data has 2-months lag

crypto_tickers = [
    "BTC-USD",  # Sep 17, 2014
    "XRP-USD",  # Sep 17, 2014
    "DOGE-USD", # Sep 17, 2014
    "ETH-USD",  # Nov 9, 2017
    "BNB-USD",  # Nov 9, 2017
    "SOL-USD",  # Apr 10, 2020
    "SHIB-USD"  # Aug 1, 2020
]
non_crypto_tickers = [
    # Indices
    "^RUT", # Russel 2000
    "^SPX", # S&P 500
    "^VIX", # CBOE Volatility Index
    "^IRX", # 13-week T-bill
    # ETFs
    "AGG",  # iShares Core U.S. Aggregate Bond ETF
    "IEF",  # iShares 7-10 Year Treasury Bond ETF
    "TLT",  # iShares 20+ Year Treasury Bond ETF
    "VNQ",  # Vanguard Real Estate ETF
    "GLD",  # SPDR Gold Trust
    "DBC",  # Invesco DB Commodity Index Tracking Fund
    "URTH"  # iShares MSCI World ETF
]

def fetch_yfinance_data(start_date, end_date):
    """
    Fetches data for crypto, ETFs, and standard indices using yfinance.
    """
    yf_tickers = crypto_tickers + non_crypto_tickers
    
    print(f"Fetching {len(yf_tickers)} tickers from Yahoo Finance...")

    try:
        data = yf.download(yf_tickers, start=start_date, end=end_date, auto_adjust=False)
        
       # Use 'Adj Close' to account for dividends/splits
        if 'Adj Close' in data:
            df = data['Adj Close']
        else:
            df = data['Close']
        
        return df
        
    except Exception as e:
        print(f"Error fetching Yahoo Finance data: {e}")
        return pd.DataFrame() # Return empty if fails

def fetch_cboe_put_data(start_date):
    """
    Fetches the specific CBOE PutWrite Index history from CBOE's direct CSV URL.
    """
    url = "https://cdn.cboe.com/api/global/us_indices/daily_prices/PUT_History.csv"
    print(f"Fetching CBOE PUT Index from {url}...")
    
    try:
        response = requests.get(url)
        response.raise_for_status() # Check for HTTP errors
        
        # Read the CSV from the response content
        df = pd.read_csv(io.StringIO(response.content.decode('utf-8')))
        
        # Standardize the date format
        df['DATE'] = pd.to_datetime(df['DATE'])
        df.set_index('DATE', inplace=True)
        
        # Filter for our start date
        df = df[df.index >= pd.to_datetime(start_date)]
        
        return df
        
    except Exception as e:
        print(f"Error fetching CBOE data: {e}")
        return pd.DataFrame() # Return empty if fails

def fetch_fama_french(start_date, end_date):
    """
    Fetches the Fama-French 3 Factors (Daily) from Kenneth French's library.
    Includes: Mkt-RF, SMB, HML, RF
    """
    print("Fetching Fama-French 3 Factors (Daily)...")
    try:
        with warnings.catch_warnings():
            warnings.simplefilter("ignore", category=FutureWarning)
            # 'F-F_Research_Data_Factors_daily' is the ID for the daily 3-factor dataset
            ds = web.DataReader("F-F_Research_Data_Factors_daily", "famafrench", start=start_date, end=end_date)
        
        # The library returns a dict
        # Key 0 typically contains the monthly returns
        # Key 1 usually contains annual returns
        df = ds[0]
        
        # FF data is usually in percentages (e.g., 0.5 means 0.5%). 
        # Optional: Divide by 100 to get decimal if needed. We will keep as raw for now.
        return df
        
    except Exception as e:
        print(f"Error fetching Fama-French data: {e}")
        return pd.DataFrame()

def fetch_cdx_data(filepath, start_date):
    """
    Fetches CDX.NA.IG data from a local CSV file.
    Assumes format: Date (DD/MM/YYYY), CDX.NA.IG
    """
    print(f"Reading CDX data from {filepath}...")
    if not os.path.exists(filepath):
        print(f"Warning: {filepath} not found.")
        return pd.DataFrame()

    try:
        # Read CSV 
        df = pd.read_csv(filepath)
        
        # Convert date column with dayfirst=True (e.g., 01/12/2015 is Dec 1st)
        df['Date'] = pd.to_datetime(df['Date'], dayfirst=True)
        
        df.set_index('Date', inplace=True)
        
        # Filter by start date
        df = df[df.index >= pd.to_datetime(start_date)]
        
        return df
        
    except Exception as e:
        print(f"Error reading CDX data: {e}")
        return pd.DataFrame()

# === Main Execution =====================================================================

# 1. Fetch Data
yf_data = fetch_yfinance_data(start_date, end_date)
cboe_data = fetch_cboe_put_data(start_date)
ff_data = fetch_fama_french(start_date, end_date)
cdx_data = fetch_cdx_data("cdx_na_ig.csv", start_date)

# 2. Merge DataFrames
print("\nMerging datasets...")
merged_df = yf_data.join(cboe_data, how='outer') \
                   .join(ff_data, how='outer') \
                   .join(cdx_data, how='outer') # 'outer' join to keep all dates (crypto trades weekends, stocks/indices don't)

# 3. Filter: Keep only dates where ALL non-crypto tickers have values
merged_df.dropna(subset=non_crypto_tickers, inplace=True)

# 4. Sort and Index by Date
merged_df.sort_index(inplace=True)
merged_df.index.name = 'DATE'

# 5. Save to CSV
filename = "historical_prices.csv"
merged_df.to_csv(filename)

print(f"Success!\nData saved to '{filename}'")
print(f"Combined data shape: {merged_df.shape}\n")
cols_to_preview = ['^SPX', 'BTC-USD', 'PUT', 'Mkt-RF', 'SMB', 'CDX.NA.IG']
available_preview = [c for c in cols_to_preview if c in merged_df.columns]
print(merged_df[available_preview].head()) # Show a few first rows
print("...")
print(merged_df[available_preview].tail()) # Show a few last rows

[*********************100%***********************]  18 of 18 completed

Fetching 18 tickers from Yahoo Finance...
Fetching CBOE PUT Index from https://cdn.cboe.com/api/global/us_indices/daily_prices/PUT_History.csv...


Fetching Fama-French 3 Factors (Daily)...
Reading CDX data from cdx_na_ig.csv...

Merging datasets...
Success!
Data saved to 'historical_prices.csv'
Combined data shape: (2220, 24)

                   ^SPX      BTC-USD      PUT  Mkt-RF   SMB  CDX.NA.IG
DATE                                                                  
2017-01-03  2257.830078  1043.839966  1677.94    0.83 -0.12     65.669
2017-01-04  2270.750000  1154.729980  1684.10    0.79  0.95     63.476
2017-01-05  2269.000000  1013.380005  1684.53   -0.21 -0.88     63.743
2017-01-06  2276.979980   902.200989  1687.89    0.29 -0.66     64.470
2017-01-09  2268.899902   902.828003  1687.42   -0.37 -0.30     65.451
...
                   ^SPX        BTC-USD      PUT  Mkt-RF   SMB  CDX.NA.IG
DATE                                                                    
2025-10-24  6791.689941  111033.921875  3244.00    0.79  0.20     51.531
2025-10-27  6875.160156  114119.328125  3253.39    1.17 -0.56     50.222
2025-10-28  6890.890137  